In [23]:
ls_columns_result = [
    'Врем. сопротивление',
    'прогноз Врем. сопротивление',
    'MAE Врем. сопротивление',
#     'MSE Врем. сопротивление',
    'Предел текучести',
    'прогноз Предел текучести', 
    'MAE Предел текучести',
#     'MSE Предел текучести',
]
ls_columns_output = [
    '№ партии',
    'марка стали',
    'диаметр',
    'толщина стенки',
    'Гр. прочн.',
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    'шаг балок закалочная печь, сек',
    'Скорость прохождения трубы через спрейер, м/с', 
    't˚ C трубы после спреера',
    '1 зона ВТР и уставка отпуск', 
    '2 зона ВТР и уставка отпуск', 
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок отпускная печь, сек',
    'C',
    'Mn',
    'Si',
    'P',
    'S',
    'Cr',
    'Ni',
    'Cu',
    'Al',
    'V',
    'Ti',
    'Nb',
    'Mo',
    'N',
    'B',
    'C-coef',
    'Параметр закалка',
    'Параметр отпуск',
    'Параметр отпуск новый V',
    'Величина зерна',
    'Тип предела текучести (1186)',
    'ICD'
    ]

ls_columns_required = [
    'марка стали',
    'диаметр',
    'толщина стенки',
    'Гр. прочн.',
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    'шаг балок закалочная печь, сек',
    'Скорость прохождения трубы через спрейер, м/с', 
    't˚ C трубы после спреера',
    '1 зона ВТР и уставка отпуск', 
    '2 зона ВТР и уставка отпуск', 
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок отпускная печь, сек',
    'Тип предела текучести (1186)'
]

In [2]:
import pandas as pd
import numpy as np
import pickle
import json
import import_ipynb
from my_libs.calc_features import *
from sklearn.preprocessing import StandardScaler
from datetime import datetime


importing Jupyter notebook from D:\Repo\PycharmProjects\gitPNTZ\PNTZ\my_libs\calc_features.ipynb


In [26]:
load = datetime.now()
database = pd.read_csv('prepared_to_saw_gp.csv')
print('База исторических режимов загружена за ', datetime.now() - load)

cleaned_input = pd.DataFrame()

                
def close_value(df, row, value):
    s_close = df[row].value_counts().index[0]
    min_delta = np.abs(s_close-value)
    for s_ in df[row].value_counts().index:
        delta = np.abs(s_-value)
        if delta<min_delta:
            min_delta = delta
            s_close = s_
    return s_close

def find_close_sort(df):
    tmp = pd.DataFrame()
    for i in range(len(df.index)):
        answ = df.iloc[i, :].copy()
        mark = answ['марка стали']
        d = answ['диаметр']
        s = answ['толщина стенки']
        gp = answ['Гр. прочн.']
        answ = database[database['Гр. прочн.'] == gp]
        if answ[answ['марка стали']==mark].shape[0]==0:
            mark = mark.split('-')[0]
        answ = answ[answ['марка стали']==mark]
        if answ[answ['диаметр']==d].shape[0]!=0:
            answ = answ[answ['диаметр']==d]
            s_close = close_value(answ, 'толщина стенки',s)
            answ = answ[answ['толщина стенки']==s_close]
        elif answ[answ['толщина стенки']==s].shape[0]!=0:
            answ = answ[answ['толщина стенки']==s]
            d_close = close_value(danswatabase, 'диаметр',d)
            answ = answ[answ['диаметр']==d_close]
        answ = answ[answ['Дата термообработки'] == answ['Дата термообработки'].max()][:1]
#         print('Исторические режимы найдены', answ.shape)
        tmp = pd.concat([tmp, answ])
        if answ.shape[0] == 0:
            print('Базовый режим для строки ', i, ' не найден.' )
            missing = ['-']
            missing = pd.DataFrame(missing)
            tmp = pd.concat([tmp, missing])
    return tmp 

# возвращает модель, список признаков, по которому строилась модель и скейлер
def load_model(dir_name, target):
    model = pickle.load(open(dir_name+'/RF_model_'+target+'.sav', 'rb'))
    ls_need_col = json.load(open(dir_name+'/ls_need_col', "r"))
#     try:
    scaler = StandardScaler()
    scale_data = json.load(open(dir_name+'/scaler', "r"))
    scaler.mean_ = scale_data[0]
    scaler.scale_ = scale_data[1]
#     except:
#         scaler=None
    return model, ls_need_col, scaler


def make_result_valid_file(file_name, dir_names, targets, output_filename):
    """Сохраняет файл с результатами """
    result = pd.DataFrame()
    base = pd.DataFrame()
    k = 0
    for target, dir_name in zip(targets,dir_names):
        model, ls_need_col, scaler = load_model(dir_name, target)
        time = datetime.now()
        if k == 0:
            try:
                valid = clean_data(file_name, ls_columns_output)
                valid.reset_index(drop=True, inplace=True)
            except KeyError:
                valid = clean_data(file_name, ls_columns_output+['№ партии', "марка стали"])
                valid.reset_index(drop=True, inplace=True)
            cleaned_input = valid.copy()
            print('Введенные данные очищены за ', datetime.now() - time)
        else:
            valid = cleaned_input.copy()
    #     print(valid['C-coef'])
    #     valid = valid[valid['C-coef']<0.6]
        time = datetime.now()
        try:
            y_valid = valid[target].copy()
        except KeyError:
            y_valid = [0 for x in range(0,valid.shape[0])]
        try:
            X_valid = valid[ls_columns_output].copy()
        except KeyError:
            X_valid = valid[ls_need_col].copy()
#         print('Магия try-except`ов за ', datetime.now() - time)
        X_valid_c = X_valid[ls_need_col].copy()
        X_valid_c = scaler.transform(X_valid_c)
        time = datetime.now()
        y_pred = model.predict(X_valid_c)
        print('Предсказания ', y_pred.shape[0], ' строчек за ', datetime.now() - time)
        result[target] = y_valid
        result['прогноз '+target] = y_pred
#         result['MAE '+target] = np.abs(result[target] - result['прогноз '+target])
        for x in range(result[target].shape[0]):
            if result.loc[x, target] != 0:
                result.loc[x, 'MAE '+target] = np.abs(result.loc[x, target] - result.loc[x, 'прогноз '+target])
            else:
                result.loc[x, 'MAE '+target] = ''
#         result['MSE '+target] = result['MAE '+target].apply(lambda x: x*x)
        k += 1

#     result['отношение'] = result[targets[0]]/result[targets[1]]
    time = datetime.now()
    base = find_close_sort(X_valid)
    print('Найдены базовые исторические режимы за ', datetime.now() - time)
    time = datetime.now()
    result = pd.concat([result, X_valid], axis=1)
#     print('Конкатенация прогноза и введенных режимов за ', datetime.now() - time)
    time = datetime.now()
    base = base[list(set(base.columns) & set(result.columns))]
    base.index = [str(i)+' исторический' for i in range(len(base.index))]
#     print('Преобразование базовых режимов за ', datetime.now() - time)
    con = pd.DataFrame()
    time = datetime.now()
    missing = ['-' for i in result.index]
    missing = pd.DataFrame(missing)
#     result.index = [str(i) for i in range(len(result.index))]
#     con = pd.concat([result, base])
    for i, j, k in zip(result.index, base.index, missing.index):
        con = pd.concat([con, result[result.index == i], base[base.index == j], missing[missing.index == k]])
#     print('Конкатенация базовых и введенных режимов за ', datetime.now() - time)
    con = con[ls_columns_result + ls_columns_output]
    con.to_excel(output_filename)
    return con, y_valid, y_pred, base

def make_result_valid_file_multi(file_name, dir_names, targets, output_filename):
    """Сохраняет файл с результатами """
    writer = pd.ExcelWriter(output_filename)
    df_dict = pd.read_excel(file_name, sheetname=None)
    for sheet, df in zip(df_dict, df_dict.values()):
        result = pd.DataFrame()
        base = pd.DataFrame()
        k=0
        for target, dir_name in zip(targets,dir_names):
            model, ls_need_col, scaler = load_model(dir_name, target)
            valid = clean_data_sheets(df, ls_need_col+['№ партии', 'марка стали'])
            y_valid = valid[target].copy()
            X_valid = valid[ls_columns_output].copy()
            X_valid_c = X_valid[ls_need_col].copy()

            y_pred = model.predict(X_valid_c)

            result[target] = y_valid
            result['прогноз '+target] = y_pred
            result['MAE '+target] = np.abs(result[target]- result['прогноз '+target])
            result['MSE '+target] = result['MAE '+target].apply(lambda x: x*x)
            X_valid = new_spr(X_valid)
            k+=1
            if k==2:
                result = pd.concat([result, X_valid], axis=1)
#                 result = pd.concat([result, base])
                result.to_excel(writer, sheet)
                
    writer.save()
    return result, y_valid, y_pred, base

# file_name = 'D:\Repo\PycharmProjects\RELEASE\PNTZ_sol_yuk\PNTZ_sol_yuk\input\input.xlsx'
file_name = 'D:\Repo\PycharmProjects\RELEASE\PNTZ_64\input\input.xlsx'
# target = 'Врем. сопротивление'
targets = ['Предел текучести','Врем. сопротивление']
dir_names = ['DATA/MODELS_RF/YS RF valid', 'DATA/MODELS_RF/H RF valid']
output_filename = 'DATA/modes_after_model/127x12,5 37g2f _ input - копия test find base modes.xlsx'

time = datetime.now()
df, y_valid, y_pred, dbasemode = make_result_valid_file(file_name, dir_names, targets, output_filename)
print('Время работы скрипта на ', int(df.shape[0]/3), 'строчках: ', datetime.now() - time)

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,3,4,5,6,7,9,10,11,14,16,17,18,19,20,21,24,25,27,28,30,32,33,34,35,36,38,39,40,41,42,44,45,46,47,49,51,53,55,56,57,58,59,60,61,63,64,65,69,70,71,72,73,76,77,78,80,81,82,83,84,85,86,87,88,90,92,93,94,97,100,101,102,103,106,107,108,109,111,113,114,116,117,119,121,123,128,129,130,133,134,135,138,139,140,141,142,143,144,146,147,150,151,152,153,155,156,160,162,166,167,168,169,170,174,176,177,179,181,182,184,186,187,189,190,191,193,194,195,196,197,200) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


База исторических режимов загружена за  0:00:00.945054
(5, 41)
(5, 41)
Введенные данные очищены за  0:00:00.239013
Предсказания  5  строчек за  0:00:00.005000
Предсказания  5  строчек за  0:00:00.003000
Базовый режим для строки  4  не найден.
Найдены базовые исторические режимы за  0:00:00.697039
Время работы скрипта на  5 строчках:  0:00:01.110063


D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\indexes\api.py:57: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [27]:
df.to_excel('to_del.xlsx')

In [37]:
input_ = pd.read_excel('D:\Repo\PycharmProjects\RELEASE\PNTZ_64\input\input___________.xlsx')
valid = clean_data(file_name, ls_columns_output)

In [ ]:
# df_dict = pd.read_excel('DATA/modes_before_model/top_typesize(рекомендации).xlsx', sheetname=None)
# for sheet, df in zip(df_dict, df_dict.values()):
#     valid = clean_data_sheets(df, ls_need_col+['№ партии', 'марка стали'])
#     print(valid)

In [10]:
def close_value(df, row, value):
    s_close = df[row].value_counts().index[0]
    min_delta = np.abs(s_close-value)
    for s_ in df[row].value_counts().index:
        delta = np.abs(s_-value)
        if delta<min_delta:
            min_delta = delta
            s_close = s_
    return s_close

def find_close_sort(df):
    tmp = pd.DataFrame()
    for i in range(len(df.index)):
        answ = df.iloc[i, :].copy()
        mark = answ['марка стали']
        d = answ['диаметр']
        s = answ['толщина стенки']
        gp = answ['Гр. прочн.']
        answ = database[database['Гр. прочн.']==gp]
        if answ[answ['марка стали']==mark].shape[0]==0:
            mark = mark.split('-')[0]
        else:
            answ = answ[answ['марка стали']==mark]
        if answ[answ['диаметр']==d].shape[0]!=0:
            answ = answ[answ['диаметр']==d]
            s_close = close_value(answ, 'толщина стенки',s)
            answ = answ[answ['толщина стенки']==s_close]
        elif answ[answ['толщина стенки']==s].shape[0]!=0:
            answ = answ[answ['толщина стенки']==s]
            d_close = close_value(danswatabase, 'диаметр',d)
            answ = answ[answ['диаметр']==d_close]
        answ = answ[answ['Дата термообработки'] == answ['Дата термообработки'].max()][:1]
#         print('Исторические режимы найдены', answ.shape)
        tmp = pd.concat([tmp, answ])
    return tmp 

In [10]:

missing

,0
0,-


In [12]:
df = pd.read_excel( 'D:\Repo\PycharmProjects\RELEASE\PNTZ_64\input\input___________.xlsx')

In [5]:
database = pd.read_csv('prepared_to_saw_gp.csv')

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,3,4,5,6,7,9,10,11,14,16,17,18,19,20,21,24,25,27,28,30,32,33,34,35,36,38,39,40,41,42,44,45,46,47,49,51,53,55,56,57,58,59,60,61,63,64,65,69,70,71,72,73,76,77,78,80,81,82,83,84,85,86,87,88,90,92,93,94,97,100,101,102,103,106,107,108,109,111,113,114,116,117,119,121,123,128,129,130,133,134,135,138,139,140,141,142,143,144,146,147,150,151,152,153,155,156,160,162,166,167,168,169,170,174,176,177,179,181,182,184,186,187,189,190,191,193,194,195,196,197,200) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
tmp = pd.DataFrame()
for i in range(len(df.index)):
    answ = df.iloc[i, :].copy()
    mark = answ['марка стали']
    d = answ['диаметр']
    s = answ['толщина стенки']
    gp = answ['Гр. прочн.']
    answ = database[database['Гр. прочн.'] == gp]

    if answ[answ['марка стали']==mark].shape[0]==0:
        mark = mark.split('-')[0]
    else:
        answ = answ[answ['марка стали']==mark]

    if answ[answ['диаметр']==d].shape[0]!=0:
        answ = answ[answ['диаметр']==d]
        s_close = close_value(answ, 'толщина стенки',s)
        answ = answ[answ['толщина стенки']==s_close]
    elif answ[answ['толщина стенки']==s].shape[0]!=0:
        answ = answ[answ['толщина стенки']==s]
        d_close = close_value(danswatabase, 'диаметр',d)
        answ = answ[answ['диаметр']==d_close]
    print(answ.shape)
    answ = answ[answ['Дата термообработки'] == answ['Дата термообработки'].max()][:1]
    print('Исторические режимы найдены', answ.shape)
    tmp = pd.concat([tmp, answ])
base = tmp[ls_columns_output]

(5667, 201)
Исторические режимы найдены (1, 201)
(5667, 201)
Исторические режимы найдены (1, 201)
(719, 201)
Исторические режимы найдены (1, 201)
(719, 201)
Исторические режимы найдены (1, 201)
(7704, 201)
Исторические режимы найдены (1, 201)


In [32]:
y_valid = tmp[targets[0]]
y_pred = tmp['predicted '+targets[0]]

from sklearn.metrics import r2_score 
from sklearn.metrics import explained_variance_score 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
print('corr',np.corrcoef(y_valid, y_pred)[0][1])
print('r2', r2_score(y_valid, y_pred))
print('explained_variance_score', explained_variance_score(y_valid, y_pred))
print('mean_squared_error', mean_squared_error(y_valid, y_pred))
print('mean_absolute_error', mean_absolute_error(y_valid, y_pred))
print('median_absolute_error', median_absolute_error(y_valid, y_pred))

d = {
    'corr': [np.corrcoef(y_valid, y_pred)[0][1]],
    'r2': [r2_score(y_valid, y_pred)],
    'explained_variance_score': [explained_variance_score(y_valid, y_pred)],
    'mean_squared_error': [mean_squared_error(y_valid, y_pred)],
    'mean_absolute_error': [mean_absolute_error(y_valid, y_pred)],
    'median_absolute_error': [median_absolute_error(y_valid, y_pred)]
}

pd.DataFrame(d).to_excel(output_filename+'_scores.xlsx')

corr 0.9491442637311868
r2 0.8930216852957163
explained_variance_score 0.8957239874637852
mean_squared_error 19.292150409068473
mean_absolute_error 3.2350262905404064
median_absolute_error 2.4166564587036916


In [31]:
y_valid = tmp[targets[1]]
y_pred = tmp['predicted '+targets[1]]

from sklearn.metrics import r2_score 
from sklearn.metrics import explained_variance_score 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
print('corr',np.corrcoef(y_valid, y_pred)[0][1])
print('r2', r2_score(y_valid, y_pred))
print('explained_variance_score', explained_variance_score(y_valid, y_pred))
print('mean_squared_error', mean_squared_error(y_valid, y_pred))
print('mean_absolute_error', mean_absolute_error(y_valid, y_pred))
print('median_absolute_error', median_absolute_error(y_valid, y_pred))

d = {
    'corr': [np.corrcoef(y_valid, y_pred)[0][1]],
    'r2': [r2_score(y_valid, y_pred)],
    'explained_variance_score': [explained_variance_score(y_valid, y_pred)],
    'mean_squared_error': [mean_squared_error(y_valid, y_pred)],
    'mean_absolute_error': [mean_absolute_error(y_valid, y_pred)],
    'median_absolute_error': [median_absolute_error(y_valid, y_pred)]
}

pd.DataFrame(d).to_excel(output_filename+'_scores.xlsx')

corr 0.9643158026543087
r2 0.9213201363358962
explained_variance_score 0.9219271770797535
mean_squared_error 11.392219003303019
mean_absolute_error 2.49501703374625
median_absolute_error 1.9771326530612328


In [93]:
for target in targets:
    print(
        df[df['MAE '+target]>5].shape[0]/df.shape[0],
          df[df['MAE '+target]>4].shape[0]/df.shape[0],
        df[df['MAE '+target]>3].shape[0]/df.shape[0])

0.20423000660938534 0.29610046265697293 0.4137475214805023
0.11103767349636484 0.18968935888962327 0.29808327825512226


In [17]:
d ={
   '% err >5': [df[df['MAE']>5].shape[0]/df.shape[0]],
   '% err >4':  [df[df['MAE']>4].shape[0]/df.shape[0]],
   '% err >3':  [df[df['MAE']>3].shape[0]/df.shape[0]]
}

pd.DataFrame(d).to_excel(dir_name+'/%.xlsx')

In [18]:
d

{'% err >3': [0.4527112232030265],
 '% err >4': [0.33921815889029006],
 '% err >5': [0.25598991172761665]}